## 1. Data preparation

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # data visualization
%matplotlib inline
from tqdm import tqdm # visualization progress bar
from zipfile import ZipFile # extract zip file

# Input data files are available in the "../input/" directory.
import os
print(os.listdir("../input"))
import shutil
import torch
import torchvision
from torchvision import transforms, models

from PIL import Image
from random import sample

In [ ]:
# let's look at the contents of the archive
train_data = ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r')
test_data = ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r')

# extract the contents
train_data.extractall()
test_data.extractall()

# let's look at the working directory
os.getcwd()

# contents of our directory
data_root = '../working'
print(os.listdir(data_root))

In [ ]:
# Define constant variable
TRAIN_DIR = 'train_dir'
VAL_DIR = 'val_dir'
TEST_DIR = 'test_dir'

batch_size = 32

# Name of classes
labels = ['dog', 'cat']

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [ ]:
# Copy test files to the unknown folder
shutil.copytree('../working/test', os.path.join(TEST_DIR, 'unknown'))

# Create train and validation folders
for dir_name in [TRAIN_DIR, VAL_DIR]:
    for label in labels:
        os.makedirs(os.path.join(dir_name, label), exist_ok=True)

# Every fifth images transfer to validation folder
for i, file_name in enumerate(tqdm(os.listdir('../working/train'))):
    dest_dir = TRAIN_DIR if i % 5 != 0 else VAL_DIR

    for label in labels:
        if file_name.startswith(label):
            shutil.copy(os.path.join('../working/train', file_name),
                        os.path.join(dest_dir, label, file_name))

In [ ]:
# visualization part of train dataset
n_samples = 20
for label in labels:
    print('Class label: ' + label)
    # path to train dataset for label
    PATH = os.path.join('/kaggle/working/train_dir', label)
    # shose random 20 images in test dir
    sub_samples = sample(os.listdir(PATH,), n_samples)
    plt.figure(figsize=(16, 12))
    for i, image in enumerate(sub_samples):
        plt.subplot(n_samples/5, 5, i+1)
        img = Image.open(os.path.join(PATH, image))
        plt.imshow(img)
        plt.title(label)
        plt.xticks([])
        plt.yticks([])
    plt.show()

In [ ]:
# visualisation part of test dataset
n_samples = 20
# shose random 20 images in test dir
sub_samples = sample(os.listdir(os.path.join('../working/test'),), n_samples)
plt.figure(figsize=(16, 12))
print('Test set')
for i, image in enumerate(sub_samples):
    plt.subplot(n_samples/5, 5, i+1)
    img = Image.open(os.path.join(os.path.join('../working/test'), image))
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
# Transform images
tt_0 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# tt_1 = transforms.Compose([
#     transforms.CenterCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std)
# ])

# tt_2 = transforms.Compose([
#     transforms.CenterCrop(224),
#     transforms.RandomRotation(degrees=45),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std)
# ])

# tt_3 = transforms.Compose([
#     transforms.CenterCrop(224),
#     transforms.RandomAffine(10, translate=[0.5, 0.3], scale=[0.7, 1.3], shear=[-10, 10]),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std)
# ])

# tt_4 = transforms.Compose([
#     transforms.RandomApply([
#         transforms.ColorJitter(
#             brightness=0.5,
#             contrast=0.5,
#             saturation=0.5,
#             hue=0.5
#         )
#     ]),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=mean, std=std)
# ])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

train_dataset_0 = torchvision.datasets.ImageFolder(TRAIN_DIR, tt_0)
# train_dataset_1 = torchvision.datasets.ImageFolder(TRAIN_DIR, tt_1)
# train_dataset_2 = torchvision.datasets.ImageFolder(TRAIN_DIR, tt_2)
# train_dataset_3 = torchvision.datasets.ImageFolder(TRAIN_DIR, tt_3)
# train_dataset_4 = torchvision.datasets.ImageFolder(TRAIN_DIR, tt_4)

val_dataset = torchvision.datasets.ImageFolder(VAL_DIR, val_transforms)

train_dataset = torch.utils.data.ConcatDataset([train_dataset_0])
#                                                 train_dataset_1,
#                                                 train_dataset_2,
#                                                 train_dataset_3,
#                                                 train_dataset_4])

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

print(len(train_dataloader), len(train_dataset))
print(len(val_dataloader), len(val_dataset))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Create function to vizualization train batch
def show_batch(X_batch, y_batch, batch_size):
    plt.figure(figsize=(16, 10))
    for i, (image_tensor, class_index) in enumerate(zip(X_batch, y_batch)):
        image = image_tensor.permute(1, 2, 0).numpy()
        image = std * image + mean
        plt.subplot(batch_size/8, 8, i+1)
        plt.imshow(image)
        plt.title(labels[class_index])
        plt.xticks([])
        plt.yticks([])
    plt.show()

for i in range(1):
    print('Batch', i+1)
    X_batch, y_batch = next(iter(train_dataloader))
    show_batch(X_batch, y_batch, batch_size)

## 2. Create neural network

In [ ]:
# Download pretrained model
model = models.resnet152(pretrained=True)

# Disable grad for all conv layers
for param in model.parameters():
    param.requires_grad = False

# Add fully connected layer
model.fc = torch.nn.Linear(model.fc.in_features, 2)

# Translate model to GPU
model = model.to(device)

# Select loss function
loss = torch.nn.CrossEntropyLoss()

# Select optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    # write train history for visualization
    train_accuracy_history = []
    train_loss_history = []
    val_accuracy_history = []
    val_loss_history = []
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}:', flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase=='train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            # compute mean loss and mean accuracy
            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            
            # write data for visualization
            if phase == 'train':
                train_accuracy_history.append(epoch_acc)
                train_loss_history.append(epoch_loss)
            else:
                val_accuracy_history.append(epoch_acc)
                val_loss_history.append(epoch_loss)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}', flush=True)

    return train_accuracy_history, train_loss_history, val_accuracy_history, val_loss_history, model

In [ ]:
%%time
history_and_trained_model = train_model(model, loss, optimizer, scheduler, num_epochs=1)

## 3. Visualization training process

In [ ]:
history = history_and_trained_model[:4]
trained_model = history_and_trained_model[-1]

In [ ]:
# # Drow graph for Accuracy and Loss on train and val subset
# titles = ['Accuracy', 'Loss']
# plt.figure(figsize=(16, 8))
# for i in range(1, 3):
#     plt.subplot(1, 2, i)
#     plt.plot(history[0+i], label='Train', c='b')
#     plt.plot(history[1+i], label='Val', c='orange')
#     plt.title(titles[i-1])
#     plt.grid()
#     plt.legend(loc='best')

In [ ]:
trained_model.eval()

for inputs, labels_ in val_dataloader:
    val_predictions = []
    val_img_paths = []
    inputs = inputs.to(device)
    labels_ = labels_.to(device)
    with torch.set_grad_enabled(False):
        preds = trained_model(inputs)
    val_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu())
    labels_ = labels_.cpu().numpy()
    predict = val_predictions[0] > 0.5
    val_predictions = val_predictions[0].numpy()
    predict = predict.numpy()
    ind = (labels_ == predict)
    
    image_tensors = []
    true_labels = []
    predicted_labels = []
    val_preds = []
    for i, image_tensor in enumerate(inputs):
        if ind[i] == False:
            image_tensors.append(image_tensor)
            true_labels.append(labels[predict[i]])
            predicted_labels.append(labels[labels_[i]])
            val_preds.append(val_predictions[i])
    
    plt.figure(figsize=(16, 16))    
    for i, image_tensor in enumerate(image_tensors):
        plt.subplot(len(image_tensors), 5, i+1)
        image = image_tensor.cpu().permute(1, 2, 0).numpy()
        image = std * image + mean
        plt.imshow(image)
        plt.title(f'True label: {true_labels[i]}\n predicted label: {predicted_labels[i]} - {val_preds[i]:.4f}')
        plt.xticks([])
        plt.yticks([])
    plt.show();

## 4. Prediction for test dataset

In [ ]:
# Modify the folder with images so that it displays not only the image with its label,
# but also the path to the image
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

test_dataset = ImageFolderWithPaths(TEST_DIR, val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
# Copy test files to the unknown folder
os.makedirs(os.path.join('../working/', 'for-test'), exist_ok=True)
shutil.copytree('../input/fortest/', '../working/for-test/for-test')

In [ ]:
test_cat_dataset = ImageFolderWithPaths('../working/for-test', val_transforms)

test_cat_dataloader = torch.utils.data.DataLoader(
    test_cat_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
# Turn model to evel mode, parametrs do not changed
trained_model.eval()

# predictions
test_predictions = []

# paths to images
test_img_paths = []

# In loop receive Batch with images, class label 'unknown' and image paths 
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        # count preds of model
        preds = model(inputs)
    # with SoftMax make probabilities
    # get the probability of the first class ('dirty plate')
    test_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    # paths needs for visualization and save preds
    test_img_paths.extend(paths)

test_predictions = np.concatenate(test_predictions)
prediction = dict(zip(test_img_paths, test_predictions))

In [ ]:
# Turn model to evel mode, parametrs do not changed
trained_model.eval()

# predictions
test_predictions = []

# paths to images
test_img_paths = []

# In loop receive Batch with images, class label 'unknown' and image paths 
for inputs, labels, paths in tqdm(test_cat_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        # count preds of model
        preds = model(inputs)
    # with SoftMax make probabilities
    # get the probability of the first class ('dirty plate')
    test_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    # paths needs for visualization and save preds
    test_img_paths.extend(paths)

test_predictions = np.concatenate(test_predictions)
prediction = dict(zip(test_img_paths, test_predictions))

In [ ]:
# Visualization predictions of classifire
plt.figure(figsize=(16, 12))
print('Visualization predictions for test set')
for i, image in enumerate(test_img_paths):
    plt.subplot(1, 2, i+1)
    img = Image.open(image)
    plt.imshow(img)
    title = 'dog' if prediction[image] > 0.5 else 'cat'
    title = f'This is a {title}: {1 - prediction[image]:.4f}'
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
plt.show()

In [ ]:
# Visualization predictions of classifire
n_samples = 25
sub_sample = sample(test_img_paths, n_samples)
plt.figure(figsize=(16, 12))
print('Visualization predictions for test set')
for i, image in enumerate(sub_sample):
    plt.subplot(n_samples/5, 5, i+1)
    img = Image.open(image)
    plt.imshow(img)
    title = 'dog' if prediction[image] > 0.5 else 'cat'
    title = f'This is a {title}: {prediction[image]:.4f}'
    plt.title(title)
    plt.xticks([])
    plt.yticks([])
plt.show()

## 5. Create  submission

In [ ]:
# Create DataFrame from paths to images and class labels
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 
                                        'label': test_predictions})
submission_df.head()

In [ ]:
# Clear id, leaving only image number and transform probabilities to classes
submission_df['label'] = submission_df['label'].map(lambda pred: 1 if pred > 0.5 else 0)
submission_df['id'] = submission_df['id'].str.replace('test_dir/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df['id'] = submission_df['id'].astype(int)
submission_df = submission_df.sort_values(by='id')
submission_df = submission_df.reset_index()
submission_df = submission_df.drop(['index'], axis='columns')
submission_df = submission_df.drop(['id'], axis='columns')
submission_df.index.name = 'id'
submission_df.index += 1
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv')

## 6. Deleting all temprary files

In [ ]:
!rm -rf train val test train_dir test_dir val_dir